<h1>
    The Variational Approximation for Bayesian Inference
</h1>
<p>
    Life after the EM Algorithm
</p>
<br>
<p>
    Dimitris G. Tzikas, Aristidis C. Likas, and Nikolaos P. Galatsanos`
</p>
<br>
<br>
<br>
<br>
<br>
<p style="font-size:10px;">CMI Lab Journal Club, 11/15/2019, Andrew Van</p>

<p>This subslide runs a bunch of scripts for this presentation.</p>
<script>window.addEventListener('load', () => { Reveal.configure({ transition: 'fade' }); });</script>

<h1>Motivation</h1>

- Variational Methods?
    - **How does Variational Expectation-Maximization work?**
    - This applies to anything with the word "Variational" (e.g. Variational Autoencoder)
    
- Succinctly...
    - Variational methods allow for the approximation of the posterior of latent variables
    - Derives a lower bound for the marginal likelihood of the observed data

<h1>Introduction</h1>

- Expectation Maximization (EM) is awesome!
    - Applications: Joint problems in image reconstruction, segmentation, registration, etc.
    - We use it to find estimates of model parameters that rely on unobserved latent variables
    - However, limited applicability when encountering complex models
    
- Variational approximation relaxes requirements of EM
    - Can be applied to a wider range of models
    - EM can be viewed as a special case of Variational EM (VEM)


<h1>Expectation-Maximization</h1>

We want to find:
$$
\hat{\theta} = \underset{\theta}{\arg\!\max} \; p(\mathbf{x}; \theta)
$$

In EM, we introduce unobserved latent variables (by marginalizing the latent variables):
$$
p(\mathbf{x};\theta) = \int p(\mathbf{x}, \mathbf{z}; \theta) d\mathbf{z}
$$
This expression is known as the **marginal likelihood**.

<h1>Evidence Lower Bound (ELBO)</h1>

$$
\ln p(x; \theta) = \int q(z) \ln p(x; \theta) dz \\
= \int q(z) \ln \Big( \frac{p(x; \theta) p(z | x; \theta)}{p(z | x; \theta)} \Big) dz \\
= \int q(z) \ln \Big( \frac{p(x, z; \theta)}{p(z | x; \theta)} \Big) dz \\
= \int q(z) \ln \Big( \frac{p(x, z; \theta) q(z)}{p(z | x; \theta) q(z)} \Big) dz \\
= \int q(z) \ln \Big( \frac{p(x, z; \theta)}{q(z)} \Big) dz - \int q(z) \ln \Big( \frac{p(z | x; \theta)}{q(z)} \Big) dz \\
= F(q, \theta) + KL(q || p)
$$